In [1]:
import numpy as np

In [2]:
%ls


Data dimensions.ipynb             losses.py
README.md                         model-training.ipynb
Untitled.ipynb                    models.py
create_cropped_imgs_acdc.py       preprocessing.ipynb
data/                             requirements.txt
dataloaders.py                    test.txt
experiment_init/                  train_model/
f1_utils.py                       treegan_workspace.code-workspace
layers_bn.py                      utils.py


In [9]:
lab_x_mini = np.load('data/gan_processed_data_original/lab_x_mini.npy')
lab_y_mini = np.load('data/gan_processed_data_original/lab_y_mini.npy')
val_x_mini = np.load('data/gan_processed_data_original/val_x_mini.npy')
val_y_mini = np.load('data/gan_processed_data_original/val_y_mini.npy')
unlab_x_mini = np.load('data/gan_processed_data_original/unlab_x_mini.npy')

In [10]:
print(lab_x_mini.shape)
print(lab_y_mini.shape)
print(val_x_mini.shape)
print(val_y_mini.shape)
print(unlab_x_mini.shape)

(20, 24, 16, 16, 16)
(20, 14, 14)
(20, 24, 16, 16, 16)
(20, 14, 14)
(30, 24, 16, 16, 16)


I am just adjusting the data Lucien created to match that of their network:
1. The multiple images stacked on top of each other is their last dimension - whereas in the npy from Lucien it is the first
    - this is something that we should fix at the data generation point this is just a patch that can be removed if Lucien regenarates the data concatenating it on the last dimension
2. I am picking one of our time dimensions
3. I am picking one of our 16 bands for now - haven't gotten to seeing how the code would handle it if I set num_channels = 16, so want to just get it to run like this first
4. I am removing a pixel around the rim - not sure why our inputs are 16x16 but our masks are 14x14, but don't want that to trip it up for now

    #L: the masks represent the labels, so our tree labels are 14x14 instead of the input dimensions of 16x16. This is something unique to our data, I'd imagine in their case (and most cases for segmentation) the dimensionality of the labels and the input data is the same. So we will have to adjust the architecture to allow for this.
    
    #L: we need to get the input back to 16x16 because all the convolutional arithmetic depends on that. will change mask to 16x16 for now and keep in mind that we need to fix that to 14x14 and address that in the GAN code.
    
    - though they have some mechanism for fixing this in utils.load_val_images, where they append 0's to match the image sizes, but not sure why their image dimensions don't match either... and that part is also commented out for now
5. we might have to adjust the size of this tiny toy dataset to be a little bit bigger
 - see my comment titled: "#D: we might have to adjust this if our mini sample is too small compared to batch size" 
     - I got rid of code that would duplicate data if the input dataset is too small, so if you get far enough in the training this might become a problem 
     - so I am letting you know here so you know that is the problem when you get to an error that might be caused by it / maybe = even better: we should just feed it a little bit of a bigger dataset. The batch_size = 20, so let's jsut feed it more than that - I think we will do that anyway, we are not connerned about proving that it works with a super tiny training sample

In [11]:
#L: todo later, we need to make sure we are selecting a single band and not a single x/y value,
#I forget how the data is originally structured
lab_x_mini2 = np.moveaxis(lab_x_mini,0,-1)[0,:,:,3,:]
val_x_mini2 = np.moveaxis(val_x_mini,0,-1)[0,:,:,3,:]
unlab_x_mini2 = np.moveaxis(unlab_x_mini,0,-1)[0,:,:,3,:]
npad = ((1, 1), (1, 1), (0, 0))
lab_y_mini2 = np.pad(np.moveaxis(lab_y_mini,0,-1), pad_width = npad, mode = 'constant')
val_y_mini2 = np.pad(np.moveaxis(val_y_mini,0,-1), pad_width = npad, mode = 'constant')

In [12]:
print(lab_x_mini2.shape)
print(lab_y_mini2.shape)
print(val_x_mini2.shape)
print(val_y_mini2.shape)
print(unlab_x_mini2.shape)

(16, 16, 20)
(16, 16, 20)
(16, 16, 20)
(16, 16, 20)
(16, 16, 30)


In [7]:
#L: Old turning everything to 14x14
#lab_x_mini = np.moveaxis(lab_x_mini,0,-1)[0,1:-1,1:-1,3,:]
#lab_y_mini = np.moveaxis(lab_y_mini,0,-1)
#unlab_x_mini = np.moveaxis(unlab_x_mini,0,-1)[0,1:-1,1:-1,3,:]

In [13]:
np.save('data/gan_processed_data/lab_x_mini.npy', lab_x_mini2)
np.save('data/gan_processed_data/lab_y_mini.npy', lab_y_mini2)
np.save('data/gan_processed_data/val_x_mini.npy', val_x_mini2)
np.save('data/gan_processed_data/val_y_mini.npy', val_y_mini2)
np.save('data/gan_processed_data/unlab_x_mini.npy', unlab_x_mini2)